<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ================================================================
# MY-AI-Gizmo • FIXED LAUNCHER (No Empty Files Bug)
# - Fixed: Does NOT create empty model files
# - Universal cleanup for broken files
# - Complete Google Drive persistence
# ================================================================

import os
import subprocess
import shutil
import re
import time
from pathlib import Path

# Colab Drive import
try:
    from google.colab import drive
except Exception:
    drive = None

# ---------------- Configuration ----------------
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")

# ---------------- Utilities ----------------
def sh(cmd, check=False):
    """Run shell command and return CompletedProcess"""
    return subprocess.run(cmd, shell=True, capture_output=True, text=True, check=check)

print("🔧 Setting up environment...")
if "MPLBACKEND" in os.environ:
    del os.environ["MPLBACKEND"]
os.environ["MPLBACKEND"] = "Agg"
print("✓ Environment configured\n")

# ═══════════════════════════════════════════════════════════════
# FIXED FUNCTION - Does NOT create empty model files!
# ═══════════════════════════════════════════════════════════════
def _ensure_drive_path(drive_path: Path, is_settings_file=False):
    """
    FIXED: Create directory structure on Drive.
    - Directories: always created
    - Settings files: can be created empty (small files)
    - Model files: NEVER pre-created (let downloads create them)
    """
    if drive_path.suffix:  # File
        drive_path.parent.mkdir(parents=True, exist_ok=True)
        # ONLY create empty files for settings, NOT models!
        if is_settings_file and not drive_path.exists():
            drive_path.touch(exist_ok=True)
    else:  # Directory
        drive_path.mkdir(parents=True, exist_ok=True)

def _remove_path(path: Path):
    """Remove file/dir/symlink at path."""
    try:
        if path.is_symlink():
            path.unlink()
        elif path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()
    except Exception:
        pass

def _create_symlink_or_fallback(src: Path, dest: Path):
    """Create symlink or fallback to copying"""
    try:
        dest.parent.mkdir(parents=True, exist_ok=True)
        if dest.exists() or dest.is_symlink():
            _remove_path(dest)
        os.symlink(str(src), str(dest), target_is_directory=src.is_dir())
        print(f"↪ linked: {dest.name}")
        return True
    except (OSError, NotImplementedError):
        try:
            if src.is_dir():
                if dest.exists():
                    _remove_path(dest)
                shutil.copytree(src, dest)
                print(f"⤷ copied directory: {dest.name}")
            else:
                dest.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(src, dest)
                print(f"⤷ copied file: {dest.name}")
            return True
        except Exception:
            return False

# ═══════════════════════════════════════════════════════════════
# UNIVERSAL CLEANUP - Removes broken files of ANY format
# ═══════════════════════════════════════════════════════════════
def cleanup_broken_files(drive_root: Path):
    """
    Remove broken model files from Drive.
    Checks: .gguf, .safetensors, .bin, .pth, .pt, .ckpt, etc.
    Deletes: Files smaller than 100KB (likely empty or broken)
    """
    print("\n🧹 Cleaning up broken files...")

    model_extensions = [
        "*.gguf", "*.safetensors", "*.bin", "*.pth", "*.pt",
        "*.ckpt", "*.h5", "*.pb", "*.onnx"
    ]

    models_dir = drive_root / "models"
    if not models_dir.exists():
        print("  No models directory yet")
        return

    broken_files = []
    for ext in model_extensions:
        for file_path in models_dir.rglob(ext):
            # Files < 100KB are probably broken
            if file_path.stat().st_size < (100 * 1024):
                broken_files.append(file_path)

    if broken_files:
        print(f"  Found {len(broken_files)} broken files:")
        for f in broken_files:
            print(f"    • {f.name} ({f.stat().st_size} bytes) - DELETING")
            try:
                f.unlink()
            except Exception as e:
                print(f"      ✗ Failed: {e}")
        print(f"  ✓ Cleaned {len(broken_files)} files")
    else:
        print("  ✓ No broken files found")

# ---------------- Start Script ----------------
print("=" * 60)
print("🚀 MY-AI-Gizmo Fixed Setup")
print("=" * 60)

# Step 1: Mount Drive
print("\n📁 Step 1/6: Mounting Google Drive...")
if drive is None:
    print("⚠️  Not in Colab, skipping mount")
else:
    try:
        drive.mount("/content/drive", force_remount=False)
        print("✓ Mounted")
    except Exception as e:
        print(f"⚠️  Mount failed: {e}")

# Step 1b: Clean broken files FIRST
cleanup_broken_files(DRIVE_ROOT)

# Step 2: Create Drive folders
print("\n💾 Step 2/6: Creating Drive structure...")
folders = [
    "models", "loras", "training", "characters", "presets", "prompts",
    "settings", "chat-history", "instruct-history", "outputs", "images",
    "logs", "cache", "extensions", "softprompts"
]
for f in folders:
    try:
        (DRIVE_ROOT / f).mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"⚠️  {f}: {e}")
print(f"✓ {len(folders)} folders ready")

# Step 3: Download repository
print("\n📥 Step 3/6: Setting up repository...")
if not WORK_DIR.exists():
    print("  Downloading...")
    sh("rm -f /content/repo.zip")
    r = sh(f"wget -q -O /content/repo.zip {REPO_ZIP}")
    if r.returncode == 0:
        sh("unzip -q /content/repo.zip -d /content")
        try:
            extracted = next(Path("/content").glob("MY-AI-Gizmo-working-*"))
            extracted.rename(WORK_DIR)
            print("✓ Downloaded")
        except StopIteration:
            print("⚠️  Extraction failed")
    else:
        print("⚠️  Download failed")
else:
    print("✓ Already exists")

if WORK_DIR.exists():
    os.chdir(WORK_DIR)

# Step 3b: Download GGUF models (FIXED - won't conflict with empty files)
print("\n📥 Step 3b/6: Downloading default GGUF models...")
MODELS_DIR = WORK_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

GGUF_MODEL_URLS = [
    "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf",
    "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    "https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
]

for url in GGUF_MODEL_URLS:
    model_name = url.split("/")[-1]
    model_path = MODELS_DIR / model_name

    # Check if file exists AND is valid (> 1MB)
    if model_path.exists() and model_path.stat().st_size > (1024 * 1024):
        print(f"  ✓ {model_name} already exists")
    else:
        # Remove broken file if exists
        if model_path.exists():
            model_path.unlink()

        print(f"  Downloading {model_name}...")
        r = sh(f"wget -q --show-progress -O '{model_path}' '{url}'")
        if r.returncode == 0:
            size_mb = model_path.stat().st_size / (1024 * 1024)
            print(f"  ✓ {model_name} ({size_mb:.1f}MB)")
        else:
            print(f"  ⚠️  Failed to download {model_name}")
            if model_path.exists():
                model_path.unlink()

# ═══════════════════════════════════════════════════════════════
# Step 4: Create symlinks (FIXED - uses new function)
# ═══════════════════════════════════════════════════════════════
print("\n🔗 Step 4/6: Linking to Google Drive...")

# Map: (local_path, drive_folder, is_settings_file)
links_map = [
    ("models", "models", False),
    ("loras", "loras", False),
    ("training", "training", False),
    ("user_data/characters", "characters", False),
    ("user_data/presets", "presets", False),
    ("user_data/prompts", "prompts", False),
    ("user_data/settings.yaml", "settings/settings.yaml", True),  # Settings
    ("user_data/settings.json", "settings/settings.json", True),  # Settings
    ("user_data/chat", "chat-history", False),
    ("user_data/instruct", "instruct-history", False),
    ("outputs", "outputs", False),
    ("user_data/images", "images", False),
    ("user_data/logs", "logs", False),
    ("user_data/cache", "cache", False),
    ("user_data/extensions", "extensions", False),
    ("user_data/softprompts", "softprompts", False),
]

for local, drive_folder, is_settings in links_map:
    drive_path = DRIVE_ROOT / drive_folder

    try:
        # FIXED: Pass is_settings_file parameter
        _ensure_drive_path(drive_path, is_settings_file=is_settings)
    except Exception as e:
        print(f"⚠️  {drive_folder}: {e}")

    local_path = WORK_DIR / local
    if local_path.exists() or local_path.is_symlink():
        _remove_path(local_path)

    local_path.parent.mkdir(parents=True, exist_ok=True)
    _create_symlink_or_fallback(drive_path, local_path)

print("✓ All data linked")

# Step 5: Local settings (fast loading)
print("\n⚙️  Step 5/6: Setting up local settings...")
drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
local_settings = WORK_DIR / "user_data" / "settings.yaml"
local_settings.parent.mkdir(parents=True, exist_ok=True)

if local_settings.is_symlink():
    try:
        local_settings.unlink()
    except Exception:
        pass

if drive_settings.exists() and drive_settings.stat().st_size > 0:
    try:
        shutil.copy2(drive_settings, local_settings)
        print(f"✓ Copied from Drive ({drive_settings.stat().st_size} bytes)")
    except Exception as e:
        print(f"⚠️  Copy failed: {e}")
        local_settings.write_text("# minimal\n")
else:
    local_settings.write_text("# minimal\nlisten: true\nshare: true\n")
    print("✓ Created minimal settings")

# Step 6: Install dependencies
print("\n📦 Step 6/6: Installing dependencies...")
print("  (First: 5-10 min, subsequent: instant)\n")

sh("chmod +x start_linux.sh")
result = sh("GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE bash start_linux.sh")

if result.returncode == 0:
    print("✓ Installation complete")
else:
    print("⚠️  Warnings (usually OK)")

# ================================================================
# LAUNCH GRADIO UI
# ================================================================

print("\n" + "=" * 70)
print("🌐 LAUNCHING GRADIO UI")
print("=" * 70)
print("\n⏳ Starting - URLs will appear below...\n")
print("=" * 70 + "\n")

sh("pkill -9 -f 'python server.py'")
sh("pkill -9 -f 'gradio'")
time.sleep(2)

env = os.environ.copy()
env["MPLBACKEND"] = "Agg"
env["GRADIO_ANALYTICS_ENABLED"] = "False"
if "PYTHONPATH" in env:
    del env["PYTHONPATH"]

python_exe = WORK_DIR / "installer_files" / "env" / "bin" / "python"
if not python_exe.exists():
    python_exe = "python3"
else:
    python_exe = str(python_exe)

cmd = f"{python_exe} -u server.py --share --listen"

print(f"🚀 Command: {cmd}\n")

proc = subprocess.Popen(
    cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1, env=env, cwd=str(WORK_DIR)
)

local_url = None
public_url = None
shown = False
error_lines = []

try:
    for line in proc.stdout:
        print(line, end="")

        if any(x in line for x in ["Error", "Exception", "Traceback"]):
            error_lines.append(line)

        if not public_url and (m := re.search(r"(https://[a-z0-9]+\.gradio\.live)", line)):
            public_url = m.group(1)

        if not local_url and (m := re.search(r"Running on local URL:\s+(http://[^\s]+)", line)):
            local_url = m.group(1)
        elif not local_url and (m := re.search(r"(http://(?:127\.0\.0\.1|0\.0\.0\.0):\d+)", line)):
            if ":5000" not in m.group(1):
                local_url = m.group(1)

        if (local_url or public_url) and not shown:
            print("\n" + "=" * 70)
            print("✨ SUCCESS! GRADIO UI RUNNING! ✨")
            print("=" * 70)

            if local_url:
                print(f"\n📍 LOCAL : {local_url}")

            if public_url:
                print(f"\n🌍 PUBLIC: {public_url}")
                print("   ↳ Click this!")

            print(f"\n💾 DRIVE : {DRIVE_ROOT}")

            print("\n" + "=" * 70)
            print("🎯 NEXT STEPS:")
            print("=" * 70)
            print("1. Click PUBLIC URL")
            print("2. Go to 'Model' tab")
            print("3. Select downloaded model")
            print("4. Click 'Load'")
            print("5. Go to 'Chat' tab")
            print("6. Start chatting!")

            print("\n" + "=" * 70)
            print("💾 AUTO-SAVED:")
            print("=" * 70)
            print("✓ Chat histories")
            print("✓ Characters")
            print("✓ Models")
            print("✓ Settings")
            print("✓ All outputs")
            print("\n💡 Re-run anytime - data persists!")
            print("=" * 70 + "\n")

            shown = True
            error_lines = []

except Exception as e:
    print(f"\n✗ Exception: {e}")

if proc.poll() is not None and not shown:
    print("\n" + "=" * 70)
    print("⚠️  SERVER STOPPED")
    print("=" * 70)
    if error_lines:
        print("\n🔍 Errors:")
        for line in error_lines[-20:]:
            print(line, end="")
    print("\n" + "=" * 70)
    print("🔧 TRY:")
    print("1. Runtime → Restart runtime")
    print("2. Re-run script")
    print("3. Check: !df -h /content/drive")
    print("=" * 70)

print("\n✓ Finished")
print(f"💾 Data: {DRIVE_ROOT}")

✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
